In [ ]:
import tensorflow as tf
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

# Создаем и обучаем модель TensorFlow
model = tf.keras.applications.ResNet50(weights='imagenet')

# Оптимизируем модель с использованием TensorRT
trt_graph = trt.create_inference_graph(
    input_graph_def=tf.graph_util.remove_training_nodes(tf.get_default_graph().as_graph_def()),
    outputs=[model.output.op.name],
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16')

# Создаем TensorFlow сессию с оптимизированным графом
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_sess = tf.Session(config=tf_config)
tf.import_graph_def(trt_graph, name='')

# Получаем входной и выходной тензоры
input_tensor = tf.get_default_graph().get_tensor_by_name('input_1:0')
output_tensor = tf.get_default_graph().get_tensor_by_name(model.output.op.name + ':0')

# Загрузим изображение и выполним инференс
import cv2
import numpy as np

image = cv2.imread('example.jpg')
image = cv2.resize(image, (224, 224))
image = np.expand_dims(image, axis=0)
output = tf_sess.run(output_tensor, feed_dict={input_tensor: image})

# Выводим результат инференса
predicted_class = np.argmax(output)
print(f'Predicted class: {predicted_class}')

# Закрываем сессию
tf_sess.close()